In [2]:
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
import time

Loading data

In [3]:
df = pd.read_csv('train_triplets.txt/train_triplets.txt',
                 delim_whitespace='\t', header=None, names=['user', 'song_id', 'play_count'])
df.head()

,user,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


Transforming play_count to 0 if user only listened to song once and 1 if listened more than once

In [4]:
df['liked'] = df['play_count'].apply(lambda x: x > 1).astype(int)
df.head()

,user,song_id,play_count,liked
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,0


Precision@K and Recall@K

In [5]:
def precision_recall_at_k(predictions, k=10, threshold=0.5, input_kind='suprise'):
    """
    Return precision and recall at k metrics for each user

    Parameters
    ----------
    predictions: Surprise Dataset or pd.DataFrame
        Predictions made from surprise dataset object or pandas dataframe
    k: int
        Number of top predictions to evaluate
    threshold: float
        Threshold for valid prediction or not
    input_kind: str
        Determines if predictions is a Surprise Data object or pandas dataframe

    Returns
    -------
        tuple(dict[user_id] = precision, dict[user_id] = recall)
            Dictionary with precision and recall for each user

    Note
    ----
    If predicting with custom input (not suprise predictions),
    data must be in the format of a 3 column pandas df or np.array
    where each sample structure is:

    [user_id, item, rating]

    Code adapted from: https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py
    """

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    if input_kind == 'suprise':
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))
    else:  # If predicting with custom input
        for uid, true_r, est in predictions:
            user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))  # Item is recommended and actually recommended
            for (est, true_r) in user_ratings[:k]  # For top k recommended items
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

Converting user-song-liked to Dataset

In [6]:
reader = Reader(rating_scale=(0,1))  #invoke reader instance of surprise library
data = Dataset.load_from_df(df[['user', 'song_id', 'liked']], reader) #load dataset into Surprise datastructure Dataset

In [66]:
start_time = time.time()
trainset, testset = train_test_split(data, test_size=0.25, random_state=123)
algo = SVD()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
print(f'Total Runtime: {time.time() - start_time}')
print(f'RMSE: {accuracy.rmse(predictions)}')

RMSE: 0.4526
RMSE: 0.4526
0.45262289221496116, runtime 1907.5774292945862


In [ ]:
precisions, recalls, ndcgs = precision_recall_at_k(predictions, k=5, threshold=0.5)

# Precision and recall can then be averaged over all users
print('Precision@5', sum(prec for prec in precisions.values()) / len(precisions))
print('Recall@5', sum(rec for rec in recalls.values()) / len(recalls))
print('NDCG@5', sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs))


precisions, recalls, ndcgs = precision_recall_at_k(predictions, k=10, threshold=0.5)

# Precision and recall can then be averaged over all users
print('Precision@10', sum(prec for prec in precisions.values()) / len(precisions))
print('Recall@10', sum(rec for rec in recalls.values()) / len(recalls))
print('NDCG@10', sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs))

precisions, recalls, ndcgs = precision_recall_at_k(predictions, k=20, threshold=0.5)

# Precision and recall can then be averaged over all users
print('Precision@20', sum(prec for prec in precisions.values()) / len(precisions))
print('Recall@20', sum(rec for rec in recalls.values()) / len(recalls))
print('NDCG@20', sum(ndcg for ndcg in ndcgs.values()) / len(ndcgs))

Precision@5: 0.35138103472656496
Recall@5: 0.26824459451601784
Precision@10: 0.3480267354097438
Recall@10: 0.31852822157947336
Precision@20: 0.34633668378417176
Recall@20: 0.34274509413820703

